In [1]:
# Import Libraries
import dash
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px

# Load Dataset
file_path = r'C:\Users\Itumeleng\Medici\sample_data.csv'  # Replace with actual file path
df = pd.read_csv(file_path)

# Preprocess Data
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Month'] = df['Date'].dt.to_period('M')
df['Year'] = df['Date'].dt.year

# Manually map CustomerLocation to coordinates
location_coords = {
    'New York': {'lat': 40.7128, 'lon': -74.0060},
    'Los Angeles': {'lat': 34.0522, 'lon': -118.2437},
    'Chicago': {'lat': 41.8781, 'lon': -87.6298},
    'Houston': {'lat': 29.7604, 'lon': -95.3698},
    'Phoenix': {'lat': 33.4484, 'lon': -112.0740},
    'San Diego': {'lat': 32.7157, 'lon': -117.1611},
    'Dallas': {'lat': 32.7767, 'lon': -96.7970},
    'San Jose': {'lat': 37.3382, 'lon': -121.8863},
    'Philadelphia': {'lat': 39.9526, 'lon': -75.1652},
}

# Add Latitude and Longitude Columns
df['Latitude'] = df['CustomerLocation'].map(lambda loc: location_coords.get(loc, {}).get('lat'))
df['Longitude'] = df['CustomerLocation'].map(lambda loc: location_coords.get(loc, {}).get('lon'))

# Handle Missing Latitude and Longitude
df['Latitude'] = df['Latitude'].fillna(0)  # Replace NaN with 0 for Latitude
df['Longitude'] = df['Longitude'].fillna(0)  # Replace NaN with 0 for Longitude

# Aggregations
monthly_revenue = df.groupby(['Date', 'Category'])['Revenue'].sum().reset_index()
location_sales = df.groupby('CustomerLocation')['Revenue'].sum().reset_index()

# Initialise Dash App
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout
app.layout = dbc.Container([
    # Header
    dbc.Row([
        dbc.Col(html.H2("Retail Data Dashboard", className="text-center text-primary"), width=12),
    ], className="mb-4"),

    # Filters Section
    dbc.Row([
        dbc.Col(html.Div([
            html.H6("Filter by Date Range"),
            dcc.RangeSlider(
                id='date-slider',
                min=df['Date'].min().year,
                max=df['Date'].max().year,
                marks={year: str(year) for year in range(df['Date'].min().year, df['Date'].max().year + 1)},
                step=1,
                value=[df['Date'].min().year, df['Date'].max().year]
            )
        ]), width=12),
    ], className="mb-4"),

    # Category Checkboxes
    dbc.Row([
        dbc.Col(html.Div([
            html.H6("Select Categories:"),
            dcc.Checklist(
                id="category-checkboxes",
                options=[{'label': cat, 'value': cat} for cat in df['Category'].unique()],
                value=df['Category'].unique().tolist(),  # Default to all categories selected
                labelStyle={"display": "block"},
                className="dcc_control",
            ),
        ]), width=12)
    ], className="mb-4"),

    # KPIs Section
    dbc.Row([
        dbc.Col(html.Div([
            html.H6("Total Revenue"),
            html.Div(id="total-revenue", className="h4")
        ], className="mini_container"), width=3),

        dbc.Col(html.Div([
            html.H6("Average Revenue per Location"),
            html.Div(id="avg-revenue-location", className="h4")
        ], className="mini_container"), width=3),

        dbc.Col(html.Div([
            html.H6("Top Category"),
            html.Div(id="top-category", className="h4")
        ], className="mini_container"), width=3),
    ], className="mb-4"),

    # Visualizations
    dbc.Row([
        dbc.Col(dcc.Graph(
            id='sales-map',
            config={'scrollZoom': True},
            style={"height": "500px"}
        ), width=8),

        dbc.Col(html.Div([
            dcc.Graph(
                id='pie-chart',
                style={"height": "400px"}
            )
        ]), width=4),
    ], className="mb-4"),

    dbc.Row([
        dbc.Col(dcc.Graph(
            id='revenue-timeseries',
            style={"height": "400px"}
        ), width=12),
    ]),
])


# Callbacks
@app.callback(
    [Output('sales-map', 'figure'),
     Output('pie-chart', 'figure'),
     Output('revenue-timeseries', 'figure'),
     Output('total-revenue', 'children'),
     Output('avg-revenue-location', 'children'),
     Output('top-category', 'children')],
    [Input('date-slider', 'value'),
     Input('category-checkboxes', 'value')]  # Get categories from checklist
)
def update_dashboard(date_range, selected_categories):
    # Filter Data
    start_year, end_year = date_range
    filtered_df = df[(df['Year'] >= start_year) & (df['Year'] <= end_year)]

    # Filter by selected categories
    if selected_categories:
        filtered_df = filtered_df[filtered_df['Category'].isin(selected_categories)]

    # Map
    map_fig = px.scatter_geo(
        filtered_df,
        lat='Latitude',
        lon='Longitude',
        color='Revenue',
        title="Sales by Location",
        size='Revenue',
        hover_data=['CustomerLocation', 'Revenue'],
        scope='usa'
    )

    # Pie Chart
    pie_fig = px.pie(filtered_df, names='Category', values='Revenue', title="Revenue by Category")
    pie_fig.update_layout(autosize=True)

    # Time Series
    timeseries_data = filtered_df.groupby('Date')['Revenue'].sum().reset_index()
    timeseries_fig = px.line(timeseries_data, x='Date', y='Revenue', title="Revenue Over Time")

    # KPIs
    total_revenue = f"${filtered_df['Revenue'].sum():,.2f}"
    avg_revenue_location = f"${filtered_df.groupby('CustomerLocation')['Revenue'].sum().mean():,.2f}"
    top_category = filtered_df.groupby('Category')['Revenue'].sum().idxmax() if not filtered_df.empty else "N/A"

    return map_fig, pie_fig, timeseries_fig, total_revenue, avg_revenue_location, top_category


# Run Server
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)